In [ ]:
# Import Libraries
import pandas as pd
from joblib import dump, load

import gc
import os
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

from numpy.random import seed
from argum import GCF

In [ ]:
# Reproducability
def set_seed(seed):
    
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(GCF.RANDOM)

In [ ]:
import pickle
from tqdm import tqdm
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler

%load_ext memory_profiler

chunk_size = GCF.CHUNK 
n_comps = GCF.COMP

%memit

with open(f'../input/split-data/IPCA_.pkl', 'rb') as f:
        
        ipca = pickle.load(f)[0]
        

FOLDS = ['FOLD_1','FOLD_2','FOLD_3','FOLD_4','FOLD_5']

for fold in FOLDS:
    
    with open(f'../input/split-data/'+ fold + '.pkl', 'rb') as f:
        
        x_tr, y_tr, x_val, y_val = pickle.load(f)
   
    x_tr = x_tr[ :, 1:]   
    x_val = x_val[ : , 1:]
    
    print('')
    print('FOLD '+str(fold))
    print('')
    print(' X_tr shape: ' +str(x_tr.shape))
    print(' X_val shape: ' +str(x_val.shape))
    print('')
    print('PCA...') 
        
    x_pca_tr = np.ndarray(shape=(0, n_comps))
    
    for i in tqdm(range(0, x_tr.shape[0]//chunk_size)):
                
        partial_x_tr = ipca.transform(x_tr[i*chunk_size : (i+1)*chunk_size])
        
        x_pca_tr = np.vstack((x_pca_tr, partial_x_tr))
        
        del partial_x_tr  
        gc.collect()
        
    x_pca_val = ipca.transform(x_val)
    
    del x_tr, y_tr, x_val, y_val
    gc.collect()
        
    pickle.dump([np.float32(x_pca_tr), np.float32(x_pca_val)] , open('PCA_'+str(fold)+'.pkl', "wb"), protocol=4)
    
    print('')
    print(' PCA_X_tr shape: ' +str(x_pca_tr.shape))
    print(' PCA_X_val shape: ' +str(x_pca_val.shape))
    
        
    del x_pca_tr, x_pca_val
    gc.collect()
    
    